## Performance analysis of classification models

Evaluating the performance of a classification model (aka 'classifier') is more complex than a regression model. With a regression model, we simply aggregate the different between the predictive value and the correct value. With classification, we need to consider the different types of errors that can occur.

In this notebook, we will analyze the performance of the classification models that we have trained in the previous notebook. We will use the following metrics:
* Accuracy
* Precision
* Recall
* F1 score

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import pickle

# set the random seed
np.random.seed(1)

## Load the previously trained model and dataset

In [2]:
log_reg_model = pickle.load(open('../data/logistic-model-smoking.pkl', "rb"))

In [3]:
df = pd.read_csv('../data/logistic-example-smoking-test.csv')
df.head(3)

,kgs_smoked,cancer
0,5.757709,0
1,8.843667,1
2,4.018594,1


In [4]:
X = df[['kgs_smoked']]
y = df[['cancer']]

In [5]:
model_preds = log_reg_model.predict(X)

In [6]:
c_matrix = confusion_matrix(y, model_preds)
c_matrix

array([[23, 11],
       [10, 36]])

In [7]:
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]


In [8]:
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * (precision * recall) / (precision + recall)

In [9]:
print(f'{"Accuracy:":12s}{accuracy:.2f}')
print(f'{"Precision:":12s}{precision:.2f}')
print(f'{"Recall:":12s}{recall:>.2f}')
print(f'{"F1:":12s}{f1:>.2f}')

Accuracy:   0.74
Precision:  0.77
Recall:     0.78
F1:         0.77


But which is the best metric to use? The answer is: it depends on the problem. 

In this case, we indicate that the cost of a test is 1000, but the cost of loosing a life is immersable (well, insurance companies try to do this - let's say it's $10,000,000). Now, from our sample population of smokers, we see that about 50% of them die of cancer. If smokers make up 10% of the population, then we can expect that 5% of the population will die of cancer. If we apply this model to the entire population (some random individual), the cost of a missed diagnosis is 10,000,000*5% = 500,000.

So, we can see that the cost of a missed diagnosis is much higher than the cost of a false positive. So, we should optimize for recall.

# Cost Weight Learning

We only know one model so far, later we will need to compare multiple models and determin which one will be our chosen model. In such cases, we need to know which score is more effective for our given context. In the above screnerio, if we give a person the test and they don't have cancer, it costs $1,000. If we don't give this person a test but they have cancer, we avoided the $1000 spent, but the cost of the life was 10,000,0000. In this particular case, we use a null model that is assume everyone has cancer and give them the test. 

Later examples will have the costs between the two errors be closer, and in such cases, a model with sufficient performance can result in cost savings and/or profit for a company.

